In [1]:
# src/build_weekly_tables.py

import pandas as pd

def build_weekly_metrics(df_reviews: pd.DataFrame) -> pd.DataFrame:
    """
    Converts review_date into weekly buckets and aggregates:
    - review_count
    - avg_rating
    """
    df = df_reviews.copy()

    df = df.dropna(subset=["review_date"])
    df["review_date"] = pd.to_datetime(df["review_date"], errors="coerce")
    df = df.dropna(subset=["review_date"])

    # Week starts Monday
    df["week"] = df["review_date"].dt.to_period("W-MON").dt.start_time

    weekly = (
        df.groupby(["app_id", "week"], as_index=False)
          .agg(review_count=("rating", "count"),
               avg_rating=("rating", "mean"))
    )

    return weekly


def add_metadata(weekly: pd.DataFrame, app_meta: pd.DataFrame) -> pd.DataFrame:
    """
    Merge app_name + category onto weekly metrics.
    app_meta must have: app_id, app_name, category
    """
    out = weekly.merge(app_meta, on="app_id", how="left")
    return out


def weekly_by_category(weekly_with_meta: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregates to category-week level.
    """
    cat = (
        weekly_with_meta.groupby(["category", "week"], as_index=False)
        .agg(review_count=("review_count", "sum"),
             avg_rating=("avg_rating", "mean"))
    )
    return cat
